# Lab | APIs

In order to use the `Spotify` API (`SpotiPy`), create an account in `Spotify` and follow [these](https://developer.spotify.com/documentation/general/guides/app-settings/) steps. 

## Authentication and initializing the API

Save your client ID and your client secret in your preferred way, and read it or load it into the following variables:

Once you have done it, we will start initializing the API.

In [196]:
import pandas as pd
import json
import spotipy
import functools
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))


## Using the search method

Now, let's use the search method by introducing a "query". For example, let's try searching for "Lady Gaga":

In [197]:
results = sp.search(q='Lady Gaga', limit=50)
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Lady+Gaga&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
       'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
       'id': '1HY2Jd0NmPuamShAr6KMms',
       'name': 'Lady Gaga',
       'type': 'artist',
       'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/4VIvfOurcf0vuLRxLkGnIG'},
       'href': 'https://api.spotify.com/v1/artists/4VIvfOurcf0vuLRxLkGnIG',
       'id': '4VIvfOurcf0vuLRxLkGnIG',
       'name': 'Bradley Cooper',
       'type': 'artist',
       'uri': 'spotify:artist:4VIvfOurcf0vuLRxLkGnIG'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'D

In [198]:
results.keys() # We can see that we only have tracks

dict_keys(['tracks'])

In [199]:
results["tracks"].keys() # Let's check the values

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [200]:
results["tracks"]["href"] # Query we have searched 

'https://api.spotify.com/v1/search?query=Lady+Gaga&type=track&offset=0&limit=50'

In [201]:
results["tracks"]["items"] #items (actual tracks)

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
     'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
     'id': '1HY2Jd0NmPuamShAr6KMms',
     'name': 'Lady Gaga',
     'type': 'artist',
     'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4VIvfOurcf0vuLRxLkGnIG'},
     'href': 'https://api.spotify.com/v1/artists/4VIvfOurcf0vuLRxLkGnIG',
     'id': '4VIvfOurcf0vuLRxLkGnIG',
     'name': 'Bradley Cooper',
     'type': 'artist',
     'uri': 'spotify:artist:4VIvfOurcf0vuLRxLkGnIG'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
   

In [202]:
results["tracks"]["limit"]#Limit we have chosen

50

In [203]:
results["tracks"]["next"] #link to the next page (next 50 tracks)

'https://api.spotify.com/v1/search?query=Lady+Gaga&type=track&offset=50&limit=50'

In [204]:
results["tracks"]["offset"] # Actual offset (starting point)

0

In [205]:
results["tracks"]["previous"] #Previous search

In [206]:
results["tracks"]["total"] # Number of matches

954

## Exploring the tracks

In [207]:
results["tracks"]["items"][0] # Explore the first song

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
    'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
    'id': '1HY2Jd0NmPuamShAr6KMms',
    'name': 'Lady Gaga',
    'type': 'artist',
    'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4VIvfOurcf0vuLRxLkGnIG'},
    'href': 'https://api.spotify.com/v1/artists/4VIvfOurcf0vuLRxLkGnIG',
    'id': '4VIvfOurcf0vuLRxLkGnIG',
    'name': 'Bradley Cooper',
    'type': 'artist',
    'uri': 'spotify:artist:4VIvfOurcf0vuLRxLkGnIG'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
  

In [208]:
results["tracks"]["items"][0].keys() # We will focus on album, artists, id, name, popularity, type and uri

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [209]:
# Track artists
results["tracks"]["items"][0]["artists"] 

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
  'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
  'id': '1HY2Jd0NmPuamShAr6KMms',
  'name': 'Lady Gaga',
  'type': 'artist',
  'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'}]

In [210]:
# Track artists names
for artist in results["tracks"]["items"][0]["artists"]:
    print(artist["name"])

Lady Gaga


In [211]:
# Track ID
results["tracks"]["items"][0]["id"] 

'2rbDhOo9Fh61Bbu23T2qCk'

In [212]:
# Track name
results["tracks"]["items"][0]["name"] 

'Always Remember Us This Way'

In [213]:
# Popularity index
results["tracks"]["items"][0]["popularity"] 

83

Spotify songs are identified by either a "url", a "uri", or an "id". 

- The `id` is an alphanumeric code, and it's the nuclear part of the identifier.

- The `uri` contains "spotify:track" before the id. An uri is useful because it can be searched manually in the Spotify app.

- The `url` is a link to the song on the Spotify web player.


In [214]:
results["tracks"]["items"][0]["uri"]

'spotify:track:2rbDhOo9Fh61Bbu23T2qCk'

## Exercise 1: Discovering New Music through Your Favorite Artists

**Objective:** 
Uncover new music by exploring the top tracks of your favorite artists and their related artists.

**Instructions:**

1. **List Your Favorite Artists**:
    - Make a list of your three favorite artists and store it in a variable named `artists`.
    - Example: `artists = ["Los Fabulosos Cadillacs", "Manu Chao", "Muchachito Bombo Infierno"]`.

2. **Fetch Top Tracks**:
    - Write a function named `get_top_tracks`.
    - This function should accept an artist's name and return the name of the first 5 top tracks by that artist.
    - Use the function `get_top_tracks` to get the first 5 top tracks for each artist in your `artists` list and store the results in a new list named `top_tracks_list`.

3. **Discover Related Artists**:
    - Write a function named `find_related_artists`.
    - This function should accept an artist's name and return the names of the first 5 artists related to the provided artist.
    - Store the results in a list named `related_artists_list`.

**Challenge:** 
Combine the above steps to create a playlist that includes the top tracks of your favorite artists and the top tracks of the artists related to them.

**Hint Section for 3. **Discover Related Artists**:**

1. **Getting Artist ID**:
    - Remember that every artist on Spotify has a unique identifier: their `id`. To get the related artists, you first need to fetch the ID of the given artist.
    - Consider using the `sp.search` method to query the artist's name. The method requires a `q` parameter, which is your query (in this case, the artist's name). It also has a `limit` parameter, which specifies the number of tracks it returns. In this case, 1 track is enough, since we just want the artist ID. 
    - Each track in the results has an associated 'artists' field. This field is a list containing details about all artists involved in that track.
   - For most tracks, especially those by a single artist, this list will contain one artist. From this artist's details, you can extract the 'id' field, which is the unique identifier for that artist on Spotify.


3. **Fetching Related Artists**:
    - Once you have the artist's ID, you can use another SpotiPy method to fetch related artists. Think about which SpotiPy method allows you to get related artists using an artist's ID. Here is the documentation link: https://spotipy.readthedocs.io/en/2.22.1/. 
    - This method will return a list of related artists. You'll need to extract the relevant details (artist names) from this list.

4. **Iterating for Multiple Artists**:
    - Once you have a function that returns related artists names for one artist, you can use a list comprehension to apply this function to a list of artist names.

5. **Testing**:
    - Always test your function with one artist name first. Once you're confident it works, then apply it to the entire list.

Remember, the key is to break the problem down into manageable steps. Use the SpotiPy documentation as a resource to understand available methods and their return structures.

In [215]:
artists = ["Lhasa De Sala", "Nouvelle Vague", "Herbie Hancock"]

In [216]:
def get_top_tracks(name, max_songs=5):
    data = sp.search(q=name)
    id = data["tracks"]["items"][0]["artists"][0]["id"]
    tracks = sp.artist_top_tracks(id)
    tracks_names = [track["name"] for track in tracks["tracks"]]
    print(tracks_names)
    return tracks_names[:max_songs]

def get_top_tracks_list(artists, n=5):
    top_tracks = {}
    for artist in artists:
        top_tracks[artist] = get_top_tracks(artist, n)
    return top_tracks

def get_related_artists(name, max_related=5):
    data = sp.search(q=name)
    id = data["tracks"]["items"][0]["artists"][0]["id"]
    related = sp.artist_related_artists(id)
    related_names = [artist["name"] for artist in related["artists"]]
    
    print(related_names)
    return related_names[:int(max_related)]

def make_playlist(artists, max_related=5, max_songs=5):
    playlist = []
    for artist in artists:
        related_artists = get_related_artists(artist, max_related)
        tracks = get_top_tracks(artist, max_songs)
        playlist.extend(tracks)
        for related in related_artists:
            tracks = get_top_tracks(related, max_songs)
            playlist.extend(tracks)
    print(playlist)
    return playlist

df = pd.DataFrame(get_top_tracks_list(artists, 5))
display(df)

make_playlist(artists, 5, 5)


['De Cara a la Pared', 'Love Came Here', 'El Desierto', 'Rising', 'Small Song', 'El Cosechero', 'La Celestina', 'Con Toda Palabra', 'La marée haute', 'Los Peces']
['In a Manner of Speaking', 'Dance With Me', 'Love Will Tear Us Apart', 'Mala Vida', 'Making Plans for Nigel', 'Ever Fallen in Love', 'The Killing Moon', "Don't Go", 'Guns of Brixton', 'Week-end à Rome']
['Cantaloupe Island - Remastered 1999/Rudy Van Gelder Edition', 'Watermelon Man - Remastered 2007', 'Watermelon Man', 'Chameleon', 'Rockit', 'Tell Me a Bedtime Story', 'Maiden Voyage', "Doin' It", 'Bring Down the Birds - Outtake', 'Paradise']


,Lhasa De Sala,Nouvelle Vague,Herbie Hancock
0,De Cara a la Pared,In a Manner of Speaking,Cantaloupe Island - Remastered 1999/Rudy Van G...
1,Love Came Here,Dance With Me,Watermelon Man - Remastered 2007
2,El Desierto,Love Will Tear Us Apart,Watermelon Man
3,Rising,Mala Vida,Chameleon
4,Small Song,Making Plans for Nigel,Rockit


['Hindi Zahra', 'Susheela Raman', 'Rosemary Standley', 'Cesária Evora', 'Moriarty', 'Alela Diane', 'Yom', 'Rokia Traoré', 'Michelle Gurevich', 'Souad Massi', 'Melissa Laveaux', 'Dom La Nena', 'Anouar Brahem', 'Ali Farka Touré', 'Sages comme des sauvages', 'Piers Faccini', 'Radio Tarifa', 'Camille', 'CocoRosie', 'Mayra Andrade']
['De Cara a la Pared', 'Love Came Here', 'El Desierto', 'Rising', 'Small Song', 'El Cosechero', 'La Celestina', 'Con Toda Palabra', 'La marée haute', 'Los Peces']
['Beautiful Tango - Unplugged', 'Imik Si Mik - Remastered Version', 'The Man I Love - Unplugged', 'Any Story', 'Beautiful Tango - Remastered Version', "Don't Forget - Unplugged", 'Un jour', 'Stand Up - Remastered Version', 'Kiss & Thrills - Remastered Version', 'Silence - Radio Edit']
['Ye Meera Divanapan Hai', 'Maya', 'Trust In Me', 'Love You Veena', 'O Rama', 'Light Years - Intro', 'Nagumomo', 'Mamavatu', 'Nagumomo - Inédit', 'What Silence Said']
['La Marelle / Amarelinha', 'Sega Jacquot', 'Au café d

['De Cara a la Pared',
 'Love Came Here',
 'El Desierto',
 'Rising',
 'Small Song',
 'Beautiful Tango - Unplugged',
 'Imik Si Mik - Remastered Version',
 'The Man I Love - Unplugged',
 'Any Story',
 'Beautiful Tango - Remastered Version',
 'Ye Meera Divanapan Hai',
 'Maya',
 'Trust In Me',
 'Love You Veena',
 'O Rama',
 'La Marelle / Amarelinha',
 'Sega Jacquot',
 'Au café du canal',
 'Wish you were here',
 'Jamaica Farewell / Sambinha',
 'Yamore',
 'When U Go - MoBlack Remix [Mixed]',
 'Clareou',
 'Shikiria - Day Mix',
 'Yamore - Edit Version',
 'Middle Of The Night',
 'P.I.M.P',
 'Balder',
 'Bad Romance - Stripped Mix',
 'Freya',
 'In a Manner of Speaking',
 'Dance With Me',
 'Love Will Tear Us Apart',
 'Mala Vida',
 'Making Plans for Nigel',
 'Koop Island Blues',
 'Strange Love',
 'Summer Sun',
 'Waltz for Koop',
 'Come to Me',
 'Beautiful Tango - Unplugged',
 'Imik Si Mik - Remastered Version',
 'The Man I Love - Unplugged',
 'Any Story',
 'Beautiful Tango - Remastered Version',
 '

## Playlists

The `sp.featured_playlists()` method in `spotipy` fetches a list of Spotify's featured playlists at a given moment. These are curated playlists that Spotify often highlights on the platform's homepage. The method provides a snapshot of the playlists that are being promoted or featured by Spotify at the time of the request.

Once you've fetched the featured playlists, you can extract their IDs (and other details).

In [217]:
featured = sp.featured_playlists() 
featured

{'message': 'Popular Playlists',
 'playlists': {'href': 'https://api.spotify.com/v1/browse/featured-playlists?offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': 'THEODORT au sommet de la première playlist de France.',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWVuV87wUBNwc'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWVuV87wUBNwc',
    'id': '37i9dQZF1DWVuV87wUBNwc',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f00000002e968819ea0f35ae14a0d7816',
      'width': None}],
    'name': 'Hits du Moment',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': '#FFFFFF',
    'public': True,
    'snapshot_id': 'Zuye4AAAAAAi4m1MeXX5Z1zqLzzxNgCf',
    'tracks': {'href': 'h

### Getting a Playlist's Details
To fetch details about a specific playlist, you can use the playlist method. You'll need the playlist's Spotify ID.

In this example, we will use the following playlist id: *37i9dQZF1DXd9zR7tdziuQ*

In [218]:
playlist_id = featured['playlists']['items'][0]['id']
playlist = sp.playlist(playlist_id)

In [219]:
print(playlist['name'])  # Print the playlist's name
print(playlist['description'])  # Print the playlist's description

Hits du Moment
THEODORT au sommet de la première playlist de France.


### Getting Tracks from a Playlist
If you want to get the tracks from a specific playlist, you can use the playlist_tracks method.

In [220]:
track_list = sp.playlist_tracks(playlist_id)
for track in track_list['items']:
    print(track['track']['name'])  # Print each track's name

Wayeh
A Bar Song (Tipsy)
Gata Only
SOIS PAS TIMIDE
It's Not Right But It's Okay
Alibi (with Pabllo Vittar & Yseult)
Nightcall
Stargazing
Passe ton chemin
Belong Together
Stumblin' In
SPIDER
The Door
Si Antes Te Hubiera Conocido
Imagine
Cry Baby
Si seulement...
Austin (Boots Stop Workin')
Move
Boucan (feat. Franglish)
I Don't Wanna Wait
Espresso
Rosa
The Sound of Silence - CYRIL Remix
Die With A Smile
Nous on sait
For Life (feat. Nile Rodgers)
Mélanine
Wake Up
Plus L'Time (Stereo Love)
Beautiful Things
Good Luck, Babe!
Position
Queen of My Castle
feelslikeimfallinginlove
Ton regard
Black Friday (pretty like the sun)
Monaco
Heaven
The Emptiness Machine
Recommence-moi
Yamore - Edit Version
Terminal 2F
BIRDS OF A FEATHER
Adieu
feelings don't lie
Taste
Summer Body
Too Sweet
Danza Kuduro - Tiësto Remix


### Getting Artists from a Playlist

To extract all the artists from the tracks in a playlist, you'd typically follow these steps:

1. Fetch the playlist's tracks.
2. Iterate through each track.
3. For each track, extract the associated artists.

In [221]:
# List to store unique artist names
artists_list = []

for track_item in track_list['items']:
    track = track_item['track']
    for artist in track['artists']:
        artist_name = artist['name']
        if artist_name not in artists_list:  # This ensures each artist is added only once
            artists_list.append(artist_name)

print(artists_list)

['THEODORT', 'Shaboozey', 'FloyyMenor', 'Cris Mj', 'GIMS', 'Mr. Belt & Wezol', 'Sevdaliza', 'Pabllo Vittar', 'Yseult', 'Kavinsky', 'Angèle', 'Phoenix', 'Myles Smith', 'Lynda', 'Mark Ambor', 'CYRIL', 'DYSTINCT', 'Teddy Swims', 'KAROL G', 'Carbonne', 'Clean Bandit', 'Anne-Marie', 'David Guetta', 'Kendji Girac', 'Dasha', 'Adam Port', 'Stryv', 'Keinemusik', 'Orso', 'Malachiii', 'KeBlack', 'Franglish', 'OneRepublic', 'Sabrina Carpenter', 'RIDSA', 'Disturbed', 'Lady Gaga', 'Bruno Mars', 'Pierre Garnier', 'Kygo', 'Zak Abel', 'Nile Rodgers', "Heuss L'enfoiré", 'Werenoi', 'Imagine Dragons', 'Léa Zelia', 'Edward Maya', 'Adryano', 'Benson Boone', 'Chappell Roan', 'Kris Kross Amsterdam', 'INNA', 'Coldplay', 'Joseph Kamel', 'Lost Frequencies', 'Tom Odell', 'Guy2Bezbar', 'Sound Of Legend', 'Linkin Park', 'SANTA', 'MoBlack', 'Salif Keita', 'Benja (NL)', 'Franc Fala', 'Cesária Evora', 'Dadju', 'Billie Eilish', 'Jérémy Frerot', 'Ofenbach', 'salem ilese', 'Helena', 'Hozier', 'Don Omar', 'Lucenzo', 'Tiës

## Exercise 2: Unraveling the World of Playlists


1. **Featured Exploration**: 
   - Fetch the list of Spotify's current featured playlists. 
   - Extract and display the names and IDs of the top 5 featured playlists.
   
2. **Deep Dive**:
   - Choose any one of the top 5 featured playlists (you can choose the one you personally find most interesting or simply pick one randomly).
   - Fetch and display its name, description, and total track count.

3. **Track-tastic**:
   - Extract and display the names of the first 10 tracks in the chosen playlist.

4. **Artistic Flair**:
   - Create a dictionary where the keys are the names of the first 10 tracks, and the values are lists containing the names of the artists associated with each track.
   - For example: `{"TrackName1": ["Artist1", "Artist2"], "TrackName2": ["Artist3"]}`
   

In [222]:
featured_playlists = sp.featured_playlists(limit=5)
playlists_info = [{'name': playlist['name'], 'id': playlist['id']}
                  for playlist in featured_playlists['playlists']['items']]
display(pd.DataFrame(playlists_info))

playlist_info = sp.playlist('37i9dQZF1DWVuV87wUBNwc')
playlist_select_infos = {
    'name': playlist_info['name'], 'description': playlist_info['description'], 'tracks_count': playlist_info['tracks']['total']}
display(pd.DataFrame(playlist_select_infos, index=[0]))

playlist_tracks = sp.playlist_tracks('37i9dQZF1DWVuV87wUBNwc')['items']
track_names = [(track['track']['name']) for track in playlist_tracks][:10]
display(pd.DataFrame(track_names))

track_artist_dict = {track['track']['name']: [artist['name'] for artist in track['track']['artists']] for track in playlist_tracks[:10]}
print(json.dumps(track_artist_dict, indent=4))



,name,id
0,Hits du Moment,37i9dQZF1DWVuV87wUBNwc
1,t'as la ref ?,37i9dQZF1DXdvyLcddcaVU
2,PVNCHLNRS,37i9dQZF1DX1X23oiQRTB5
3,Best Life,37i9dQZF1DWYVURwQHUqnN
4,TIAKOLA : BDLM VOL. 1,0qylIR3cDeMuvyltTYMZmH


,name,description,tracks_count
0,Hits du Moment,THEODORT au sommet de la première playlist de ...,50


,0
0,Wayeh
1,A Bar Song (Tipsy)
2,Gata Only
3,SOIS PAS TIMIDE
4,It's Not Right But It's Okay
5,Alibi (with Pabllo Vittar & Yseult)
6,Nightcall
7,Stargazing
8,Passe ton chemin
9,Belong Together


{
    "Wayeh": [
        "THEODORT"
    ],
    "A Bar Song (Tipsy)": [
        "Shaboozey"
    ],
    "Gata Only": [
        "FloyyMenor",
        "Cris Mj"
    ],
    "SOIS PAS TIMIDE": [
        "GIMS"
    ],
    "It's Not Right But It's Okay": [
        "Mr. Belt & Wezol"
    ],
    "Alibi (with Pabllo Vittar & Yseult)": [
        "Sevdaliza",
        "Pabllo Vittar",
        "Yseult"
    ],
    "Nightcall": [
        "Kavinsky",
        "Ang\u00e8le",
        "Phoenix"
    ],
    "Stargazing": [
        "Myles Smith"
    ],
    "Passe ton chemin": [
        "Lynda"
    ],
    "Belong Together": [
        "Mark Ambor"
    ]
}
